In [1]:
import pandas as pd
import time 
import pickle
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

#for RNN
#import tensorflow as tf
#from sklearn.model_selection import train_test_split
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
#import pre-processed data from pickle 
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2011_text_wo_names.pickle")
#df_10= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2010_text_wo_names.pickle")
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2009_text_wo_names.pickle")
#df_12= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2012_text_wo_names.pickle")
df_13= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2013_text_wo_names.pickle")
df_13

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
9,"[rate, increase, indication, important, rat, f...",0,1,[],None,"""The rate of increase is an indication of just...",0,2013,1
13,"[wildlife, many, year, although, rat, always, ...",1,0,[],None,"""Lundy has been a wildlife haven for many year...",0,2013,0
17,"[spend, day, come, song, every, day, year]",1,0,[],None,He spent each day of 2012 coming up with a son...,19,2013,0
21,"[asked, get, inspiration, daily, basis, reply,...",3,0,[],None,"""Asked how he got his inspiration on a daily b...",19,2013,0
22,"[emotion, day, sometimes, pick, paper, sometim...",1,0,[],None,"""The emotion I was having those days and somet...",19,2013,0
...,...,...,...,...,...,...,...,...,...
3480510,"[year, old, deal, label, mean, bring, first, s...",2,0,[],None,The 40-year-old's deal with the label means he...,2168370,2013,0
3480514,"[say, excitedly, partner, year, felt, time, co...",1,0,[Rob],None,"Columbia Records Chairman Rob Stringer said: ""...",2168370,2013,0
3480519,"[start, produce, career, one, half, release, f...",1,0,"[Pharrell, Neptunes]",True,Pharrell started his producing career as one h...,2168370,2013,0
3480530,"[speaking, year, old, fan, say, fly, home, fam...",0,2,[],None,"Speaking to Newsbeat, 16-year-old fan Opal-Ros...",2169203,2013,1


In [3]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [4]:
#apply function to only get rows with an absolute count 
df_13['col_type'] = df_13.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_13= df_13[df_13["col_type"].notnull()]

#DOC: number of male and female columns
df_13["col_type"].value_counts()  

0.0    673481
1.0    217465
Name: col_type, dtype: int64

**Define TFIDF Vectorizer**

In [5]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [ ]:
#word2vec to dos
#import gensim
#from gensim.models import Word2Vec
#file_w2vec= (r"/Users/yolandaferreirofranchi/Desktop/GoogleNews-vectors-negative300.bin") #yolanda's path 
#model_w2v = gensim.models.KeyedVectors.load_word2vec_format(file_w2vec, binary=True)

In [6]:
probabilities = []

In [7]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg', 'sag' 'saga'], #removed liblinear as it is for small + medium datasets & NOT for sparse data
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_clf = LogisticRegression(**grid_search.best_params_)

    #based on GridsearchCV
    #penalty = 'l2'
    #solver = 'lbfgs'
    #C = 100
    #class_weight = 'balanced'
    #random_state = 42 

    #best_clf = LogisticRegression(penalty=penalty, C= C, solver = solver, class_weight = class_weight, random_state = random_state)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    #results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Accuracy: {accuracy:.2f}")
    #print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:200]
    low_coef = sorted_coef[-200:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])


    #write word probability list
    feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    positive_probas = probas[:, 1]

    feature_prob_dict = {}
    for feature, index in feature_indices.items():
        proba = (X_transformed[:, index].toarray() * positive_probas).mean()
        feature_prob_dict[feature] = proba
        probabilities.append(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    #df_probs_top = df_probs.sort_values('probability', ascending=False)[:200]
    
    #save model 
    with open('results.pkl', 'wb') as f:
        pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef, best_clf, class_report #df_probs_top

**Coefficient Analysis**

In [8]:
df_probs_13 = logistic_regression_year(df_13, 'col_type', 'pre_processed_sent')
df_probs_13

: 

: 

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [ ]:
#open model performance metrics 
with open('results_13.pkl', 'rb') as f:
    results = pickle.load(f)

accuracy = results['accuracy']
report = results['report']

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification report:\n{report}")

In [ ]:
#create DF of highest coef
highest_coef_12 = pd.DataFrame(df_probs_12[0]) 

#create DF of lowest lowest coef manipulation 
lowest_coef_12 = pd.DataFrame(df_probs_12[1]) 
lowest_coef_12 = lowest_coef_12.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
highest_coef_12.head(50)

In [ ]:
#save DF as pickle file per year 
lowest_coef_12.to_pickle('RESULTS12_coef_low.pickle')
highest_coef_12.to_pickle('RESULTS12_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Word Probability Analysis** 

#make sure to un-comment the last two lines of the functions

In [ ]:
#access list of top words 
word_prob_10 = pd.DataFrame(df_probs_10[3]) 
word_prob_10 = word_prob_10.sort_values(["probability"], ascending= False)

#save list of words (highest to lowest probability of being female) for year: 
word_prob_10.to_pickle('RESULTS09_words_prob.pickle')
word_prob_10.head(10)

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

**Support Vector Machine**

In [ ]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
def svm_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'C': [.001, .01, .1, 1, 10, 100],
        #'kernel': default('rbf')',
        'degree': [2,3,4],
        'gamma': ['scale', 'auto'],
        #'class_weight': [None],
        #'random_state': [42]
    }

    #the classifier 
    clf = SVC()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = SVC(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Year: {year}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #get important features (equivalent of coefficients in logreg)
    result_clf = permutation_importance(best_clf, X_test, y_test, n_repeats=10, random_state=42)
    importance_scores = result_clf.importances_mean
    importance_scores_df = pd.DataFrame(importance_scores, columns=['tbd'])

    for i in range(len(importance_scores)):
        print("Feature {}: Importance score = {:.3f}".format(i, importance_scores[i]))
    
    #print probability results 
    #feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    #probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    #positive_probas = probas[:, 1]

    #print(f"\nProbability of the Following Words Being Female:")
    #feature_prob_dict = {}
    #for feature, index in feature_indices.items():
        #proba = (X_transformed[:, index] * positive_probas).mean()
        #feature_prob_dict[feature] = proba
        #print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return importance_scores_df

In [ ]:
svm_year(df_09, 'col_type', 'pre_processed_sent') #takes more than 1 hour to run

In [ ]:
#save the SVM model 
import joblib
model, accuracy, class_report = svm_year(df_09, 'col_type', 'pre_processed_sent')

# Save the model and performance metrics
joblib.dump(model, 'model_svm.pkl')
with open('performance_metrics.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Classification Report:\n{class_report}")

**Comparing the Models**

In [ ]:
model_results = {' ': ['Logistic Regression', 'SVM', 'NN'],
           'Accuracy': [0.66, 0.70, 0],
           'Time (s)': [54.39, 3540, 0]}

# Create a pandas dataframe from the dictionary
df_mod_results = pd.DataFrame(model_results)

# Set the index of the dataframe to the Kernel column
df_mod_results.set_index(' ', inplace=True)

# Display the dataframe
print(df_mod_results)